<a href="https://colab.research.google.com/github/pvanik1/FastAI/blob/master/Lesson_2_Homework_4_types_of_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!curl -s https://course.fast.ai/setup/colab | bash;
from fastai.vision import *

In [0]:
mkdir /root/.fastai/data/cars

In [0]:
cd /root/.fastai/data/cars

In [0]:
from google.colab import files
files.upload()

In [0]:
classes = ['bmw','ford','honda','mercedes']

In [0]:
for c in classes:
  print(c)
  folder = c
  file = 'urls_' + c + '.txt'
  path = Path('/root/.fastai/data/cars')
  dest = path/folder
  dest.mkdir(parents=True, exist_ok=True)
  download_images(path/file, dest, max_pics=200)
  verify_images(path/c, delete=True, max_size=500)

In [0]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [0]:
data.classes
data.show_batch(rows = 3, figsize = (7,8))

In [0]:
data.classes, data.c, len(data.valid_ds), len(data.train_ds)

In [0]:
learn = create_cnn(data, models.resnet34, metrics = error_rate)

In [0]:
learn.fit_one_cycle(4)

In [0]:
learn.save('stage-1')

In [0]:
learn.unfreeze()
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(4, max_lr=slice(1e-04,1e-03))

In [0]:
learn.save('stage-2a')

In [0]:
learn.load('stage-2a');
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
interp.plot_confusion_matrix()

In [0]:
interp.most_confused(min_val=4)

In [0]:
interp.plot_top_losses(8)

In [0]:
learn.export()

In [0]:
defaults.device = torch.device('cpu')

In [0]:
img = open_image(path/'honda'/'00000008.png')
img

In [0]:
learn = load_learner(path)

In [0]:
pred_class,pred_idx,outputs = learn.predict(img)
pred_class